<a href="https://colab.research.google.com/github/srmt99/stock-market/blob/master/corr_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mount google drive** (colab only)


In [0]:
# run this cell only when using colab
from google.colab import drive
from os.path import join
ROOT = '/content/drive'
print(ROOT)
drive.mount(ROOT, force_remount=True)

%mkdir '/content/drive/My Drive/stock-market'
%cd '/content/drive/My Drive/stock-market'
%mkdir 'data/'
%mkdir 'all_records/'
%ls

%tensorflow_version 2.x
%load_ext tensorboard

# **Import libraries**

In [0]:
import glob
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from tensorflow import keras
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import glob
import os
import copy
import random as rnd
import datetime
from pathlib import Path
mpl.rcParams['figure.figsize'] = (6, 4)
mpl.rcParams['axes.grid'] = False

# **Hyperparameters**

In [0]:
# data extraction
plt.figure(figsize=(10,6))
test_split = 0.2
future_prediction = 1
corr_w = 40
w = 30
val_split, test_split = 0.1, 0.2
batch_size = 32

# **Download data** (colab only)

In [0]:
# uploading the data from github into colab's working space
%cd 'data'
!wget https://github.com/srmt99/stock-market/blob/master/data/markets.npy?raw=true -O markets.npy
!wget https://github.com/srmt99/stock-market/blob/master/data/stocks_1.npy?raw=true -O stocks_1.npy
!wget https://github.com/srmt99/stock-market/blob/master/data/stocks_2.npy?raw=true -O stocks_2.npy
%cd ..
%ls

# **Reading stocks and markets**

In [0]:
stocks = []
for i in np.load("data/stocks_1.npy",allow_pickle=True):
    stocks.append(i)
for i in np.load("data/stocks_2.npy",allow_pickle=True):
    stocks.append(i)
stocks = np.array(stocks)
markets = np.load("data/markets.npy",allow_pickle=True)
                  
# extracting minimum market length
min_market_len = len(markets[0])
for i in markets[1:]:
    if len(i)<min_market_len:
        min_market_len = len(i)

# making all markets of the same length
for i in range(len(markets)):
    while len(markets[i])>min_market_len:
        markets[i] = np.delete(markets[i],0,0)

for i in range(len(stocks)):
    while len(stocks[i])>min_market_len:
        stocks[i] = np.delete(stocks[i],0,0)

markets = np.stack(markets,0)

# **Plot random stock and market**

In [0]:
# ploting some random stock prices and markets
r1 = np.random.randint(len(stocks))
r2 = np.random.randint(len(markets))
print(r1,r2)
plt.figure(figsize=(20,6))
plt.plot(stocks[r1][:,1],label="stock prices")
plt.plot(markets[r2][:,1],label="market")
plt.legend()
plt.show()

# **Utility functions**

In [0]:
def turn_into_windows(input,kernel):
    """
    this function is used to make windows out of the input array and select a label for each window
    input array of shape: (len_input, num_features) is turned into windows of shape: (kernel,num_features)
    and a labels is selected for each window , which is a floating point number label is selected
    from the second features (arr[...,1]) which here is <CLOSE> price for the NEXT sample
    NOTE: every window and it's label is normilized together and the info for denormilizing it is stored in
    norm_info, which at this time is not needed therfore is commented.

    input: 2D array of (time_samples, features)
    kernel: window size

    returns: two np.arrays of shape: (len_input - kernel + 1, kernel, num_features) , (len_input - kernel + 1,)
    """
    data = []
    labels = []
    # norm_info = []
    # j = 0
    # j is for counting num(windows) , used for norm_info
    for i in range(kernel,len(input)-1):
        window = input[i-kernel:i+1,:]
        # norm_info.append([np.min(window,0),None])
        window = window - np.min(window,0)
        # norm_info[j][1] = np.max(window,0)+0.0001
        window = window / (np.max(window,0)+0.0001)
        data.append(window[:-1,:])
        labels.append(window[-1,1])
        # j+=1
    return np.array(data),np.array(labels).reshape(len(labels),1)

def turn_into_windows_multi(input,kernel):
    """
    this function is mush similar to the one above (turn_into_windows)
    yet this fucntion was created speceficaly for extracting 2d matrices from all markets as windows.
    meaning at each step, we make <num_markets> windows from all market <CLOSE> prices together, which
    will increase the speed of creating the records further.
    input is a 2d matrix of shape: (num_markets, len_markets) and is turned into 2d matrices
    of shape: (num_markets, kernel)
    NOTE: all of the data in each matrix of windows, is normilized. the info is stored in norm_info
    which is commented at the time being , since there is no use for it.

    input: 2d matrix of shape: (num_markets, time_samples)
    kernel: window size

    returns: a np.array of shape: (len_markets - kernel + 1, num_markets, kernel)
    """
    data = []
    norm_info = []
    # j = 0
    # j is for counting num(windows) , used for norm_info
    for i in range(kernel,input.shape[1]-1):
        window = input[:,i-kernel:i]
        # norm_info.append([np.min(window,1).reshape((len(window),1)),None])
        window = window - np.min(window,1).reshape((len(window),1))
        # norm_info[j][1] = np.max(window,1).reshape((len(window),1))+0.0001
        window = window / (np.max(window,1).reshape((len(window),1))+0.0001)
        data.append(window)
    return np.array(data)

def smooth(input):
    output = []
    output.append(input[0])
    output.append(np.mean([input[0],input[1]]))
    for i in range(2,len(input)-2):
        mean = np.mean([input[i-2],input[i-1],input[i],input[i+1],input[i+2]])
        output.append(mean)
    output.append(np.mean([input[len(input)-2],input[len(input)-1]]))
    output.append(input[len(input)-1])
    return output

# **Building and saving records**

In [0]:
"""
this part, creates the records and saves them as np.array into files
it itreates over stocks, one stock at a time, the stock is first turend into windows
of shape: (corr_w,) holding <CLOSE> price in a list of windows. these windows are used
to calculate the correlation with the coresponding market windows of shape: (num_mrkets,corr_w)
meaning the correalation of each stock window is calculataed against all markets at the same time
then, both the stock and markets are turned into windows of length <w> to be put into the records
togrther with the before calculated correlations.
"""
for count,stock in enumerate(stocks):
    records = []
    labels = []
    correlations = []
    x,y = turn_into_windows(stock,corr_w)
    for wc,window in enumerate(x[:,:,1]):
        corr = np.corrcoef(window,markets[: , min_market_len - len(x) - corr_w - 1 + wc : min_market_len - len(x) + wc -1 , 1])[1:,0]
        correlations.append( np.nan_to_num(corr) )
    correlations = np.array(correlations)
    x,y = turn_into_windows(stock,w)
    x2 = turn_into_windows_multi(markets[:,:,1],w)
    for wc in range(len(correlations),w,-1):
        record = np.zeros( (2*len(markets)+5,w) )
        record[:5,:] = np.transpose(x[wc + (len(x)-len(correlations)-1) ][:,1:]) # part 1
        record[5:5+len(markets)] = np.transpose(correlations[wc-w:wc]) # part 2
        record[5+len(markets):5+2*len(markets)] = x2[wc + (len(x2)-len(correlations)-1)] # part 3
        records.append(record)
        labels.append(y[wc + (len(x)-len(correlations)-1) ])
    np.save(f"all_records/records_{count}",np.array([records,labels]))
    print(f"{count}/{len(stocks)}")
print("__done__")

# **Reading records**

In [0]:
train = []
labels = []
for count,filename in enumerate(glob.glob("all_records/records_*.npy")):
    x,y = np.load(filename,allow_pickle=True)
    for i in x:
        train.append(i)
    for i in y:
        labels.append(i[0])
    if count == 300:
        break

train = np.array(train)
labels = np.array(labels)

# **Train, validation and test data preparation**

In [0]:
""" 
We split data into 3 parts: train, validation, test
We control the number of records in each set using
test_split and val_split
"""
data = train
num_val, num_test = int(val_split * data.shape[0]), int(test_split * data.shape[0])

train_x = data[:-(num_val + num_test)]
train_y = labels[:-(num_val + num_test)]
val_x = data[-(num_val + num_test):-num_test]
val_y = labels[-(num_val + num_test):-num_test]
test_x = data[-num_test:]
test_y = labels[-num_test:]

train_x[np.isnan(train_x)] = 0
train_y[np.isnan(train_y)] = 0
val_x[np.isnan(val_x)] = 0
val_y[np.isnan(val_y)] = 0
test_x[np.isnan(test_x)] = 0
test_y[np.isnan(test_y)] = 0

print('train_x:', train_x.shape)
print('train_y:', train_y.shape)
print('val_x:', val_x.shape)
print('val_y:', val_y.shape)
print('test_x:', test_x.shape)
print('test_y:', test_y.shape)

# **Building tf.data.Dataset from numpy arrays**

In [0]:
"""
In this cell we build tf.data.Dataset
from numpy arrays
"""
train = tf.data.Dataset.from_tensor_slices((train_x, train_y))
train = train.shuffle(1024).batch(batch_size).repeat()
print('train dataset built')
val = tf.data.Dataset.from_tensor_slices((val_x, val_y))
val = val.shuffle(1024).batch(batch_size).repeat()
print('validation dataset built')
test = tf.data.Dataset.from_tensor_slices((test_x, test_y))
test = test.shuffle(1024).batch(batch_size)
print('test dataset built')

# **Callback definitions**

In [0]:
"""
Here is the defintion of some callbacks
"""

# tensorboard callback
log_dir = Path('./logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# checkpoint callback
checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath='./checkpoints/model_{epoch}',
                                                      save_best_only=True,
                                                      monitor='val_loss',
                                                      verbose=0)
# earlystopping callback
earlystopping_callback = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                                          min_delta=0.005,
                                                          patience=3,
                                                          restore_best_weights=True,)

# TerminateOnNaN callback
terminateonNaN_callback = tf.keras.callbacks.TerminateOnNaN()

# **Building and training model**

In [0]:
"""
Building and training a model
"""
# model params
lstm1_out = 50
dense1_out = 30
dense2_out = 1
num_epoch = 10
alpha = 5e-4
reg = 0

! rm -r logs # delete any logs so far
model = keras.Sequential([
  keras.layers.LSTM(lstm1_out, input_shape=train_x[0].shape, kernel_regularizer=keras.regularizers.l1_l2(l1=reg,l2=reg)),
  keras.layers.Dense(dense1_out, activation='relu', kernel_regularizer=keras.regularizers.l1_l2(l1=reg,l2=reg)),
  keras.layers.Dense(dense2_out, activation='sigmoid', kernel_regularizer=keras.regularizers.l1_l2(l1=reg,l2=reg))
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=alpha),
              loss='mse',
              metrics=['mse', 'mae'])
history = model.fit(train, validation_data=val, validation_steps=10, epochs=num_epoch, steps_per_epoch=2324,
                    callbacks=[tensorboard_callback,
                               checkpoint_callback,
                               earlystopping_callback,
                               terminateonNaN_callback])

print("evaluating on test set")
t = model.evaluate(test,verbose=1)

# **Saving model parameters and results**

In [0]:
f = open("results.txt","a")
loss = history.history["loss"][-1]
val_loss = history.history["val_loss"][-1]
string = f"l_o={lstm1_out},d1_o={dense1_out},d2_o={dense2_out},e={num_epoch},a={alpha},r={reg}:l={str(loss)[:6]},v_l={str(val_loss)[:6]},t_l={str(t[0])[:6]}\n"
f.write(string)
f.close()

# **Launch TensorBoard**

In [0]:
logs_base_dir = "./logs"
%tensorboard --logdir {logs_base_dir}

# **Plot random validation and train data predictinos**

In [0]:
"""
In this section we randomly plot the results of
predictions on 1 example from both validation and test sets. 
"""

val_sample = rnd.randint(1, val_x.shape[0] - 1)
x1 = val_x[val_sample, :, :].reshape(1, val_x.shape[1], -1)
y1 = val_y[val_sample]
train_sample = rnd.randint(1, train_x.shape[0] - 1)
x2 = train_x[train_sample, :, :].reshape(1, train_x.shape[1], -1)
y2 = train_y[train_sample]
test_sample = rnd.randint(1, test_x.shape[0] - 1)
x3 = test_x[test_sample, :, :].reshape(1, test_x.shape[1], -1)
y3 = test_y[test_sample]

yprime1 = model.predict(x1)
yprime2 = model.predict(x2)
yprime3 = model.predict(x3)

x_axis = [i for i in range(w + 1)]
fig, a = plt.subplots(3, 1)
fig.set_figheight(8)
fig.set_figwidth(8)
a[0].plot(x_axis[:-1], x1[0, 0, :], 'bo', label='history')
a[0].plot(x_axis[-1:], y1, 'ro', label='real')
a[0].plot(x_axis[-1:], yprime1, 'go', label='prediction')
a[0].legend()
a[0].set_title('Validation Set')
a[1].plot(x_axis[:-1], x2[0, 0, :], 'bo', label='history')
a[1].plot(x_axis[-1:], y2, 'ro', label='real')
a[1].plot(x_axis[-1:], yprime2, 'go', label='prediction')
a[1].legend()
a[1].set_title('Train Set')
a[2].plot(x_axis[:-1], x3[0, 0, :], 'bo', label='history')
a[2].plot(x_axis[-1:], y3, 'ro', label='real')
a[2].plot(x_axis[-1:], yprime3, 'go', label='prediction')
a[2].legend()
a[2].set_title('Test Set')